# Importar drivers

## Os drivers estão todos localizados na pasta Instruments

In [1]:
from instruments.M8195A_driver import * # Driver do AWG
from instruments.ATS9872_driver import * # Placa Alazar

## A biblioteca de medida está localizada na pasta measurements.

In [2]:
from measurements.DataChannelManager import * # Contém classe que gerencia os dados para serem carregados no AWG
from measurements.PulseSequence import * # Classe que estratura e sequencia os pulsos

# Diferentes tipos de pulsos que podem ser usados

from measurements.waveforms.SquarePulse import *
from measurements.waveforms.SquareSideBandPulse import *
from measurements.waveforms.ZeroPulse import *
from measurements.waveforms.GaussianPulse import *
from measurements.waveforms.DragPulse import *

# Biblioteca dos Outros Instrumentos

In [3]:
# drivers de controle dos instrumentos
from instruments.ATS9872_driver import * # alazar placa de aquisição
from instruments.DG645_driver import * # delay generator 
from instruments.M8195A_driver import * # awg
from instruments.E8257D_driver import * #RF source
from instruments.E5063A_driver import * #VNA
from instruments.Agilent11713C_driver import * # attenuator
from instruments.SIM928_driver import * # Voltage source
from instruments.SIM970_driver import * # Voltage meter

## Outras bibliotecas para ajudar na medida

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
import struct
from IPython.display import clear_output
from datetime import datetime

# Configura os outros Instrumentos

In [5]:
#att_address='TCPIP0::169.254.101.101::inst0::INSTR'
#RFsourceMeasurement_address='TCPIP0::169.254.101.103::inst0::INSTR'
RFsourceExcitation_address='TCPIP0::169.254.101.104::inst0::INSTR'
#Voltsource_address='GPIB0::2::INSTR'
#Voltsource_channel=8
#Voltmeter_address='GPIB0::2::INSTR'
#Voltmeter_channel=5
#vna = E5063A_driver("TCPIP0::169.254.101.105::inst0::INSTR")
# alazar, placa de aquisição
#inst_att = Agilent11713C_driver(att_address)
#inst_RFsourceMeasurement = E8257D_driver(RFsourceMeasurement_address)
inst_RFsourceExcitation = E8257D_driver(RFsourceExcitation_address)
#inst_voltsource =  SIM928_driver(Voltsource_address,Voltsource_channel,step_time=0.1,step_voltage=0.001)
#inst_voltmeter = SIM970_driver(Voltmeter_address,Voltmeter_channel)

In [6]:
# inst_att = Agilent11713C_driver(att_address)

In [7]:
# inst_att.get_attenuation()

## Configura as fontes

In [8]:
# temos que configurar a fonte no modo de pulsos com trigger externo
# a polaridade escolhida da fonte depende do canal utilizado do awg.
# todo canal do awg é duplicado. Ele vai ter um canal normal e outro que idêntico ao normal só que invertido.
# A fonte de medida recebe o invertido enquando a placa alazar recebe o normal. Eu faço isso para
# manter o nível do trigger igual em ambos.
#inst_RFsourceMeasurement.set_pulse_trigger_external()
#inst_RFsourceMeasurement.setPulsePolarityInverted()
#inst_RFsourceMeasurement.start_mod()


# A fonte de excitação recebe o canal 3 - marker 1 -  normal
inst_RFsourceExcitation.set_pulse_trigger_external()
inst_RFsourceExcitation.setPulsePolarityNormal()
inst_RFsourceExcitation.start_mod()

# Configuramos amplitude do sinal da fonte de medida. Ela tem que ser o suficiente para alimentar os mixers que heterodinagem.
# calculamos que seu valor deve ser 18 dBm.
#inst_RFsourceMeasurement.set_amplitude(18)

# A fonte de excitação não é necessariamente usada no processo de medida da cavidade (a menos que se queira ver os deslocamento dispersivo de frequência)
# no caso sua amplitude é por volta de 11 dBm
inst_RFsourceExcitation.set_amplitude(11)

In [9]:
# parar as fontes
#inst_RFsourceMeasurement.stop_rf()
inst_RFsourceExcitation.stop_rf()

## Configurar AWG

In [10]:
inst_awg = M8195A_driver('169.254.101.100')

inst_awg.setSampleRate(64e9)

# tem que conectar também o awg a nossa referência de rubídio 10 MHz
inst_awg.setRefInClockExternal()  
inst_awg.setRefInClockFrequency(10e6)

# configura o awg para usar dois canais (um de excitação, outro de medida) e usar dois markers
# o canal 1 gerará o sinal de excitação I e o sinal de medida
# o canal 2 gerará o sinal de excitação Q
# o canal 3 será o marker 1 - para ligar a fonte 2, fonte de excitação
# o canal 4 será o marker 2 - para ligar a fonte 1, de medida, e para trigar a placa de aquisição alazar
inst_awg.setDualWithMarker()

# dividir a memória em 2
inst_awg.setMemoryDivision(2)
# usar a memória externa para o canal 2
inst_awg.setChannelMemoryToExtended(2)


# markers should have 1 V amplitude and offset 0.5
inst_awg.setVoltage(3,1) 
inst_awg.setVoltage(4,1)

inst_awg.setVoltageOffset(3,0.5) 
inst_awg.setVoltageOffset(4,0.5)

inst_awg.enableChanneloutput(1)
inst_awg.enableChanneloutput(2)
inst_awg.enableChanneloutput(3)
inst_awg.enableChanneloutput(4)

In [11]:
inst_awg.getError()

In [12]:
inst_awg.stop()

In [22]:
1/70e6*1e9, 1/240e6*1e9

(14.285714285714286, 4.166666666666667)

## Configurar DataChannel

In [18]:
ms = DataChannelManager(inst_awg)

ms.clearAwgChannel()

ms.labelAwgChannel(channel = 2, # o canal do awg
                        channelName = 'Q', markerValue = 1)

ms.labelAwgChannel(channel = 1, # o canal do awg
                        channelName = 'I', markerValue = 1)

ms.labelAwgChannel(channel = 1, # o canal do awg
                        channelName = 'm', markerValue = 2)

In [24]:
SquareSideBandPulse?

Init signature: SquareSideBandPulse(length, frequency, amplitude=1)
Docstring:      This class defines a pulse to be sent to the AWG.
File:           c:\users\franscisco rouxinol\developer\measurementlfdq\src\measurements\waveforms\squaresidebandpulse.py
Type:           ABCMeta
Subclasses:     

In [16]:
import time

https://github.com/Rouxinol-Research-Lab/MeasurementsLFDQ/blob/main/src/instruments/DSOS604A_driver.py

In [20]:
#para equipamentos
inst_awg.stop()
inst_RFsourceExcitation.stop_rf()
start=time.time()
inst_RFsourceExcitation.set_amplitude(11)
inst_RFsourceExcitation.set_frequency(3.5e9)
print("bfr",time.time()-start)
inst_RFsourceExcitation.query("*OPC?")
print("aft",time.time()-start)

# Define-se um pulso Zero
p1 = SquarePulse(length = 5e-9, amplitude =0.5)
#p1 = GaussianPulse(length = 1e-7, amplitude =1,sigma=1e-8)
#p1=SquareSideBandPulse(length=300e-6, frequency=100e6, amplitude=1)


s1 = PulseSequence('Twotone', 10e-9)

s1.clear()

s1.add(p = p1, channel = 'I', delay = 0)

p2 = SquarePulse(length = 5e-9, amplitude =1)
s1.add(p = p2, channel = 'Q', delay = 0)

#start=time.time()
ms.prepareChannelData(sequence = s1, totalExperimentDuration = 500e-6)
#print("bfr",time.time()-start)
#inst_awg.query("*OPC?")
#print("aft",time.time()-start)

sleep(2)
inst_awg.freeMemory()
sleep(0.5)
ms.allocAwgMemory()
sleep(0.5)
ms.loadChannelDataToAwg('I')
sleep(0.5)
ms.loadChannelDataToAwg('Q')
sleep(0.5)
ms.setInstrumentsMarker(3)
sleep(0.5)

#sleep(1)
inst_awg.getError()
inst_RFsourceExcitation.start_rf()
inst_awg.start()

bfr 0.004988431930541992
aft 0.02593374252319336


In [67]:
#10-13dB
#3-7GHz
inst_RFsourceExcitation.set_amplitude(10)
inst_RFsourceExcitation.set_frequency(3e9)
inst_RFsourceExcitation.start_rf()

VisaIOError: VI_ERROR_CONN_LOST (-1073807194): The connection for the given session has been lost.

In [35]:
inst_awg.start()

In [59]:
inst_awg.stop()
inst_RFsourceExcitation.stop_rf()

In [65]:
inst_RFsourceExcitation.stop_rf()
inst_RFsourceExcitation.set_amplitude(11)
inst_RFsourceExcitation.set_frequency(6e9)
inst_RFsourceExcitation.start_rf()